
## Planning Algorithms in Gridworld</h2>

- ### Value Iteration

In [ ]:
from typing import Tuple
import jax.numpy as jnp
import jax.random as jrd
from nestedtuple import nestedtuple

import jaxdp
from jaxdp.planning.algorithms import value_iteration
from jaxdp.planning.runner import train, no_update_state


# Define the arguments
@nestedtuple
class Args:
    seed: int = 42

    class train_loop:
        gamma: float = 0.99
        n_iterations: int = 100
        verbose: bool = False

    class value_init:
        minval: float = 0.0
        maxval: float = 1.0

    class mdp_init:
        p_slip: float = 0.1
        board: Tuple[str] = ["#####",
                             "#  @#",
                             "#  X#",
                             "#P  #",
                             "#####"]


args = Args()
# Initiate the MDP and the Q values
key = jrd.PRNGKey(args.seed)
mdp = jaxdp.mdp.grid_world(**args.mdp_init._asdict())
init_value = jrd.uniform(key, (mdp.action_size, mdp.state_size,),
                         dtype="float32", **args.value_init._asdict())

# Define value update function and its initial state
update_state = None
update_fn = no_update_state(value_iteration.update.q)

# Train a policy
metrics, value, update_state = train(
    mdp=mdp,
    init_value=init_value,
    update_state=update_state,
    update_fn=update_fn,
    value_star=jnp.zeros_like(init_value),
    **args.train_loop._asdict()
)

- ### Visualize

In [ ]:
from itertools import chain
import pandas as pd
from plot_util import make_figure


# Make dataframe from the metrics
percentile = 25

index = pd.MultiIndex.from_product(
    [["gridworld"], ["VI"], list(range(100))],
    names=["ENV", "ALG", "STEP"])
columns = pd.MultiIndex.from_product(
    [metrics._fields, ["low", "med", "high"]],
    names=["METRIC", "PERCENTILE"])

data = []
for name in metrics._fields:
    values = getattr(metrics, name)
    if values.ndim == 1:
        values = values.reshape(1, -1)
    percentiles = jnp.nanpercentile(
        values, q=jnp.array([percentile, 50, 100 - percentile]), axis=0)
    data.append(percentiles)


df = pd.DataFrame(data=jnp.stack(list(chain(*data)), axis=1), columns=columns, index=index)

# Generate the figure
make_figure(df.loc["gridworld"])